# About

flowCore: Basic structures for flow cytometry data
```
Provides S4 data structures and basic functions to deal with flow cytometry data.
```

In [3]:
library(tidyverse)
library(flowCore)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.8
✔ tidyr   0.8.2     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks flowCore::filter(), stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
flowFrame

function (exprs, parameters, description = list()) 
{
    if (!is.matrix(exprs) || !is.numeric(exprs) || is.null(colnames(exprs))) 
        stop("Argument 'exprs' must be numeric matrix with colnames ", 
            "attribute set", call. = FALSE)
    if (missing(parameters)) 
        parameters <- parDefault(exprs)
    else isValidParameters(parameters, exprs)
    checkClass(description, "list")
    new("flowFrame", exprs = exprs, parameters = parameters, 
        description = description)
}
<environment: namespace:flowCore>

In [63]:
###
col_name = LETTERS[1:5]

###
exprs    = rnorm(100) %>% matrix(., nrow = 20, ncol = 5)
colnames(exprs) = col_name

### range of variables
tmp = apply(exprs, 2, range) %>% 
    t %>% data.frame %>% 
    `colnames<-`(c("minRange", "maxRange")) %>%
    mutate(range = maxRange - minRange)

### metadata: dataframe about variables
df = data.frame(name = col_name, desc = paste("var", col_name))
df = bind_cols(df, tmp)

### metadata of metadata
metadata = data.frame(labelDescription = c(
    "Name of Parameter", 
    "Description of Parameter", 
    "Minimum Parameter Value", 
    "Maximum Parameter Value", 
    "Range of Parameter"))

### create annotated dataframe based on df and metadata
ann = AnnotatedDataFrame(data = df, varMetadata = metadata)

### create flowFrame
flowFrame(exprs, parameters = ann)

flowFrame object 'anonymous'
with 20 cells and 5 observables:
  name  desc  minRange maxRange    range
1    A var A -2.647159 1.352082 3.999241
2    B var B -1.861482 2.669977 4.531459
3    C var C -1.469309 1.641277 3.110586
4    D var D -2.129754 2.526904 4.656658
5    E var E -1.257401 2.748884 4.006285
0 keywords are stored in the 'description' slot

In [13]:
data(GvHD)
GvHD

A flowSet with 35 experiments.

An object of class 'AnnotatedDataFrame'
  rowNames: s5a01 s5a02 ... s10a07 (35 total)
  varLabels: Patient Visit ... name (5 total)
  varMetadata: labelDescription

  column names:
  FSC-H SSC-H FL1-H FL2-H FL3-H FL2-A FL4-H Time

In [14]:
frame = GvHD[[1]]
frame

flowFrame object 's5a01'
with 3420 cells and 8 observables:
     name              desc range minRange maxRange
$P1 FSC-H        FSC-Height  1024        0     1023
$P2 SSC-H        SSC-Height  1024        0     1023
$P3 FL1-H         CD15 FITC  1024        1    10000
$P4 FL2-H           CD45 PE  1024        1    10000
$P5 FL3-H        CD14 PerCP  1024        1    10000
$P6 FL2-A              <NA>  1024        0     1023
$P7 FL4-H          CD33 APC  1024        1    10000
$P8  Time Time (51.20 sec.)  1024        0     1023
153 keywords are stored in the 'description' slot

In [17]:
exprs(frame) %>% head

FSC-H,SSC-H,FL1-H,FL2-H,FL3-H,FL2-A,FL4-H,Time
371,396,2432.871983,507.887297,18.156914,110,21.739192,1
190,62,7.513726,1006.775298,26.982678,213,1.000000,1
141,197,3.194470,597.239582,3.109343,132,29.524716,1
167,265,1977.824185,143.998435,5.839470,28,4.579326,1
128,30,1.321941,1.640793,5.632914,0,1.655632,1
208,60,8.295949,1121.639700,10.868463,240,1.000000,1


In [18]:
parameters(frame)

An object of class 'AnnotatedDataFrame'
  rowNames: $P1 $P2 ... $P8 (8 total)
  varLabels: name desc ... maxRange (5 total)
  varMetadata: labelDescription

AnnotatedDataFrame {Biobase}
```
Class Containing Measured Variables and Their Meta-Data Description.
```
An AnnotatedDataFrame consists of two parts. There is a collection of **samples** and the values of **variables** measured on those samples. There is also a **description** of each variable measured. The components of an AnnotatedDataFrame can be accessed with **pData** and **varMetadata**.

In [28]:
param = parameters(frame)
param@dimLabels
pData(param)
varMetadata(param)

[1] "rowNames"    "columnNames"

,name,desc,range,minRange,maxRange
$P1,FSC-H,FSC-Height,1024,0,1023
$P2,SSC-H,SSC-Height,1024,0,1023
$P3,FL1-H,CD15 FITC,1024,1,10000
$P4,FL2-H,CD45 PE,1024,1,10000
$P5,FL3-H,CD14 PerCP,1024,1,10000
$P6,FL2-A,NA,1024,0,1023
$P7,FL4-H,CD33 APC,1024,1,10000
$P8,Time,Time (51.20 sec.),1024,0,1023


,labelDescription
name,Name of Parameter
desc,Description of Parameter
range,Range of Parameter
minRange,Minimum Parameter Value after Transformation
maxRange,Maximum Parameter Value after Transformation


In [34]:
df <- data.frame(x = 1:6,
                 y = rep(c("Low", "High"),3),
                 z = I(LETTERS[1:6]),
                 row.names = paste("Sample", 1:6, sep = "_"))

metaData <- data.frame(labelDescription=c("Numbers","Factor levels","Characters"))

In [35]:
df

,x,y,z
Sample_1,1,Low,A
Sample_2,2,High,B
Sample_3,3,Low,C
Sample_4,4,High,D
Sample_5,5,Low,E
Sample_6,6,High,F


In [36]:
metaData

labelDescription
Numbers
Factor levels
Characters


In [41]:
ann = Biobase::AnnotatedDataFrame(data = df, varMetadata = metaData)
ann

An object of class 'AnnotatedDataFrame'
  rowNames: Sample_1 Sample_2 ... Sample_6 (6 total)
  varLabels: x y z
  varMetadata: labelDescription

In [43]:
pData(ann)
varMetadata(ann)

,x,y,z
Sample_1,1,Low,A
Sample_2,2,High,B
Sample_3,3,Low,C
Sample_4,4,High,D
Sample_5,5,Low,E
Sample_6,6,High,F


,labelDescription
x,Numbers
y,Factor levels
z,Characters
